<a href="https://colab.research.google.com/github/ZhongdiChu/CouseraDL.AI/blob/main/GAPigmentsTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
tf.test.gpu_device_name()
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Mounted at /content/drive
  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=5a463696c82ade5e27ba11ea5b1dfb431f234a18796067d0c1844996d9451601
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.2 GB  | Proc size: 1.1 GB
GPU RAM Free: 14852MB | Used: 227MB | Util   2% | Total 15079MB


In [5]:
from keras import backend as K
import random
import os
import numpy as np
from skimage.io import imsave, imread
from skimage.transform import resize

imgs = np.load('/content/drive/MyDrive/Colab/RGBCC_img.npy')
imgs_mask = np.load('/content/drive/MyDrive/Colab/imgs_mask_RGB_train.npy')


img_oh = np.copy(imgs_mask[:,:,:,0:2])

# for index in range(0,53):

#   jk = imgs_mask[index,::]
#   jkm = 255-np.max(jk,axis = 2)
#   img_oh[index,:,:,2] = jkm


random.Random(1294).shuffle(imgs)
random.Random(1294).shuffle(img_oh)
imgs_train = imgs[:40,:]
imgs_mask_train  = img_oh[:40,:]
imgs_test = imgs[40:,:]
imgs_mask_test = img_oh[40:,:]

# imgs_train = preprocessRGB(imgs_train)
# imgs_mask_train = preprocess(imgs_mask_train)
imgs_train = imgs_train.astype('float32')
imgs_mask_train = imgs_mask_train.astype('float32')
imgs_mask_train = imgs_mask_train/255.  # scale masks to [0, 1]
imgs_train = imgs_train/255.  # scale masks to [0, 1]

In [8]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, Dropout, UpSampling2D, Reshape, Activation
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


def unet():

    inputs = Input((int(512),int(512),int(3)))
    num_classes = 2


    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    
    conv9 = Conv2D(num_classes, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    
    conv10 = Conv2D(num_classes, 1, activation = 'sigmoid', padding = 'same',)(conv9)
    # out = Reshape((512*512,num_classes))(conv10)
    # final_out = Activation("softmax")(out)

    model = Model(inputs=[inputs], outputs=[conv10])

    print(model.summary())



    

    # model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.compile(optimizer=Adam(learning_rate=5e-5), loss=dice_cost_12, metrics=[dice_coef12])

    # print(model.summary())

    # if(pretrained_weights):
    # 	model.load_weights(pretrained_weights)

    return model

# def dice_coef(y_true, y_pred):
#     y_true_f = tf.cast(K.flatten(y_true), tf.float32)
#     y_pred_f = K.flatten(y_pred)
#     intersection = K.sum(y_true_f * y_pred_f)
#     return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)



def dice_cost_12(y_true, y_predicted):
    
    return -dice_coef12(y_true, y_predicted)

def dice_coef12(y_true, y_predicted):

    mask_true1 = K.flatten(y_true[:, :, :,  0])#
    mask_pred1 = K.flatten(y_predicted[:, :, :, 0])#

    num_sum1 = 2.0 * K.sum(mask_true1 * mask_pred1) + K.epsilon()
    den_sum1 = K.sum(mask_true1) + K.sum(mask_pred1)+ K.epsilon()

    mask_true2 = K.flatten(y_true[:, :, :, 1])#
    mask_pred2 = K.flatten(y_predicted[:, :, :, 1])#

    num_sum2 = 2.0 * K.sum(mask_true2 * mask_pred2) + K.epsilon()
    den_sum2 = K.sum(mask_true2) + K.sum(mask_pred2)+ K.epsilon()

    return (num_sum1/den_sum1)/2 + (num_sum2/den_sum2)/2




# def gen_dice(y_true, y_pred, eps=1e-6):
#     """both tensors are [b, h, w, classes] and y_pred is in logit form"""

#     # [b, h, w, classes]
#     pred_tensor = tf.nn.softmax(y_pred)
#     y_true_shape = tf.shape(y_true)

#     # [b, h*w, classes]
#     y_true = tf.reshape(y_true, [-1, y_true_shape[1]*y_true_shape[2], y_true_shape[3]])
#     y_pred = tf.reshape(pred_tensor, [-1, y_true_shape[1]*y_true_shape[2], y_true_shape[3]])

#     # [b, classes]
#     # count how many of each class are present in 
#     # each image, if there are zero, then assign
#     # them a fixed weight of eps
#     counts = tf.reduce_sum(y_true, axis=1)
#     weights = 1. / (counts ** 2)
#     weights = tf.where(tf.math.is_finite(weights), weights, eps)

#     multed = tf.reduce_sum(y_true * y_pred, axis=1)
#     summed = tf.reduce_sum(y_true + y_pred, axis=1)

#     # [b]
#     numerators = tf.reduce_sum(weights*multed, axis=-1)
#     denom = tf.reduce_sum(weights*summed, axis=-1)
#     dices = 1. - 2. * numerators / denom
#     dices = tf.where(tf.math.is_finite(dices), dices, tf.zeros_like(dices))
#     return tf.reduce_mean(dices)
    
# def dice_coef_loss(y_true, y_pred):
#     return -gen_dice(y_true, y_pred)

In [10]:
model = unet()


model.load_weights('/content/drive/MyDrive/Colab Notebooks/2classweights.h5')

model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)
    
history = model.fit(imgs_train, imgs_mask_train, batch_size=8, epochs=500, verbose=1, shuffle=True,
               validation_split=0.2,
               callbacks=[model_checkpoint])


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_48 (Conv2D)              (None, 512, 512, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_49 (Conv2D)              (None, 512, 512, 64) 36928       conv2d_48[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_8 (MaxPooling2D)  (None, 256, 256, 64) 0           conv2d_49[0][0]                  
____________________________________________________________________________________________